In [97]:
import numpy as np
import matplotlib.pyplot as plt

In der Datenstruktur geht es um veschiedere Wetterstationen in diesem Fall die Wetterstationen von `Bern`. 

Die Spaltennamen werden hier unten klar definiert. 


|  Header   |        Bedeutung        |  Einheit   |
| :-------: | :---------------------: | :--------: |
|    stn    |      Wetterstation      |     -      |
|   time    |       Zeitstempel       | `datetime` |
| prestah0  |        Luftdruck        |    hPa     |
| tre200h0  |       Temperatur        |     °C     |
| rre150h0  |       Regenmenge        |     mm     |
| su2000h0  |      Sonnenstunden      |     %      |
| time_diff | Stunden seit Messbeginn |     h      |

Schon am Anfang beginnen die ersten Probleme aufzutauchen, denn wie wir wissen fehlen gewisse Daten in der Datenstruktur. Um die möglichkeit frei zu behalten zu einem späteren Zeitpunkt fehlender Werte zu ergänzen oder zu filtern haben wir uns für die funktion `genfromtxt` entschieden, denn diese ermöglicht es uns fehlende Werte zu filtern oder zu ergänzen 

In [98]:
# Hier hat man genfromtxt() benutzt, um die Daten aus der CSV-Datei zu laden.
data = np.genfromtxt("Bern21-22.csv", delimiter=';', skip_header=1, dtype='str')

In einem nächsten Schritt werden die explizite Daten der Wetterstation in **Bern** gefiltert. Gleich danach wird das Array Transponiert um die Zugewiesenen Variablen den richtigen Spalten zuzuweisen. 

Eine reduktion von **52'324** auf **7464** Daten kann nach diesem Prozess beobachtet werden. Trotz der gefilterten Daten bleibt  die Zeilenanzahl enthalten. 


In [99]:
# Daten nach BER filtern
ber_data = data[np.char.startswith(data[:,0], "BER")]

# Variablen gemäss Aufgabenstellung zuweisen
stn, time, luftdruck, temp, regen, sonne, time_diff = ber_data.T

# shape und len() der Arrays ausgeben
print(f" Der ursprüngliche Datensatz betägt {data.shape}, {ber_data.shape}")
print(len(data),len(ber_data))

# Die ersten 5 Zeilen ausgeben
print(ber_data[:5])

 Der ursprüngliche Datensatz betägt (52324, 7), (7464, 7)
52324 7464
[['BER' '2021-11-01 00:00:00' '941.4' '7.2' '0.0' '0.0' '0.0']
 ['BER' '2021-11-01 01:00:00' '941.4' '7.5' '0.0' '0.0' '1.0']
 ['BER' '2021-11-01 02:00:00' '941.1' '8.3' '0.7' '0.0' '2.0']
 ['BER' '2021-11-01 04:00:00' '941.3' '9.6' '1.0' '0.0' '4.0']
 ['BER' '2021-11-01 06:00:00' '942.2' '9.6' '1.7' '0.0' '6.0']]


Jedoch wollen wir ersteinmal wissen, wieviele Daten, dass überhaupt fehlen, damit wir uns ein Bild machen können. Wir wissen, dass die Wetterdaten über ein Jahr gesammelt worden sind und dass ein Tag aus 24h besteht. 

In [100]:
datensätze = 365 * 24
unsere_datensätze = len(ber_data)
print(f"Es fehlen insgesammt {datensätze - unsere_datensätze} Datensätze")

Es fehlen insgesammt 1296 Datensätze


In [101]:
"""import pandas as pd
# Interpolationspunkte festlegen
x = ber_data["time_diff"]
y_cols = ["luftdruck", "Temp", "regen", "sonne"]

# Interpolation durchführen
interp_data = np.broadcast({"time_diff": np.arange(x.min(), x.max()+1)})
for col in y_cols:
    interp_func = np.interp(interp_data["time_diff"], x, data[col])
    interp_data[col] = interp_func

# Ergebnis ausgeben
print(interp_data)
"""

'import pandas as pd\n# Interpolationspunkte festlegen\nx = ber_data["time_diff"]\ny_cols = ["luftdruck", "Temp", "regen", "sonne"]\n\n# Interpolation durchführen\ninterp_data = np.broadcast({"time_diff": np.arange(x.min(), x.max()+1)})\nfor col in y_cols:\n    interp_func = np.interp(interp_data["time_diff"], x, data[col])\n    interp_data[col] = interp_func\n\n# Ergebnis ausgeben\nprint(interp_data)\n'

In [102]:
#BER;2021-11-01 00:00:00;941.4;7.2;0.0;0.0;0.0
#BER;2022-11-01 23:00:00;959.1;9.6;0.0;0.0;8783.0

Interpolation 

In [132]:
import numpy as np

def remove_ber(data):
    # entferne die Spalte "BER"
    updated_data = np.delete(data, 0, axis=1)
    return updated_data


# aktualisiertes Array ohne die Spalte "BER"
updated_data = remove_ber(ber_data)
# aktualisierte Spaltennamen ohne "BER"
updated_data[:, 2:] = updated_data[:, 2:].astype(float)


updated_data

array([['2021-11-01 00:00:00', '941.4', '7.2', '0.0', '0.0', '0.0'],
       ['2021-11-01 01:00:00', '941.4', '7.5', '0.0', '0.0', '1.0'],
       ['2021-11-01 02:00:00', '941.1', '8.3', '0.7', '0.0', '2.0'],
       ...,
       ['2022-11-01 21:00:00', '958.6', '11.1', '0.0', '0.0', '8781.0'],
       ['2022-11-01 22:00:00', '958.9', '10.0', '0.0', '0.0', '8782.0'],
       ['2022-11-01 23:00:00', '959.1', '9.6', '0.0', '0.0', '8783.0']],
      dtype='<U19')

In [104]:
import pandas as pd
def read_ber_data(filename):
    # Einlesen der CSV-Datei
    df = pd.read_csv(filename, header=None, sep=';')
    
    # Filtern nach Datensätzen mit Präfix "BER"
    df = df[df[0].str.startswith('BER')]
    
    # Spaltennamen zuweisen
    df.columns = ['Station', 'Datum', 'Druck', 'Temp', 'Regen', 'Schnee', 'Wind']
    
    return df


In [105]:
df = read_ber_data('Bern21-22.csv')

In [106]:
df

,Station,Datum,Druck,Temp,Regen,Schnee,Wind
7446,BER,2021-11-01 00:00:00,941.4,7.2,0.0,0.0,0.0
7447,BER,2021-11-01 01:00:00,941.4,7.5,0.0,0.0,1.0
7448,BER,2021-11-01 02:00:00,941.1,8.3,0.7,0.0,2.0
7449,BER,2021-11-01 04:00:00,941.3,9.6,1.0,0.0,4.0
7450,BER,2021-11-01 06:00:00,942.2,9.6,1.7,0.0,6.0
...,...,...,...,...,...,...,...
14905,BER,2022-11-01 18:00:00,958.1,11.1,0.0,0.0,8778.0
14906,BER,2022-11-01 19:00:00,958.3,10.9,0.0,0.0,8779.0
14907,BER,2022-11-01 21:00:00,958.6,11.1,0.0,0.0,8781.0
14908,BER,2022-11-01 22:00:00,958.9,10.0,0.0,0.0,8782.0
